In [5]:
import os
import requests
import re
from googlesearch import search 
from tabula import read_pdf
import urllib.parse
import pandas as pd
from datetime import datetime, timedelta
import pdfplumber
from IPython.core.debugger import set_trace
from itertools import combinations
import sys, traceback

dirName = "announcements"
dirPath = ".\\" + dirName + "\\"

try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  announcements  already exists


In [2]:
def download_file(url):
    file = requests.get(url, stream=True)
    headers = file.headers['content-disposition']
    filename = re.findall("filename=\"(.+)\"", headers)[0]
    location = os.path.relpath(dirPath + filename)
    open(location, 'wb').write(file.content)
    del file

In [15]:
query = "\"S&P 500 INDEX\" \"Set to Join\" site:spindices.com filetype:pdf"
results = search(query, tld='com', lang='en', num=10, start=0, stop=None, pause=10.0)

In [16]:
for result in results:
    try:
        download_file(result)
    except:
        traceback.print_exc(file=sys.stdout)
    print(result)

https://us.spindices.com/documents/indexnews/announcements/20200331-1119193/1119193_1utxspinrtn15otiscarrier.pdf
https://www.spindices.com/documents/indexnews/announcements/20190917-998359/998359_totalsystemstss5cdw.pdf
https://us.spindices.com/documents/indexnews/announcements/20200506-1138565/1138565_dexdomino456capristor.pdf?force_download=true
https://www.spindices.com/documents/indexnews/announcements/20190314-892576/892576_21foxspindis100adbe.pdf
https://us.spindices.com/documents/indexnews/announcements/20200518-1145759/1145759_jcpenney6helmer.pdf?force_download=true
https://www.spindices.com/documents/indexnews/announcements/20200122-1077359/1077359_5payc.pdf
https://www.spindices.com/documents/indexnews/announcements/20191127-1041559/1041559_BerkleyRJoe546.pdf
https://www.spindices.com/documents/indexnews/announcements/20190801-969367/969367_1leidos546foot6fizz6.pdf
https://www.spindices.com/documents/indexnews/announcements/20191118-1035566/1035566_Thermo1Service5.pdf
https:/

https://www.spindices.com/documents/indexnews/announcements/20150309-152351/152351_petskyjet546.pdf
http://www.spindices.com/documents/indexnews/announcements/20160324-323162/323162_healthnetfirsten524pom425holxmigr46.pdf
http://www.spindices.com/documents/indexnews/announcements/20160301-312347/312347_524slhcnxawk.pdf
https://us.spindices.com/documents/indexnews/announcements/20131211/67155_fbshuf252426100.pdf
Traceback (most recent call last):
  File "C:\Users\Yingchen\Anaconda3\envs\finance\lib\site-packages\urllib3\connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "C:\Users\Yingchen\Anaconda3\envs\finance\lib\site-packages\urllib3\util\connection.py", line 84, in create_connection
    raise err
  File "C:\Users\Yingchen\Anaconda3\envs\finance\lib\site-packages\urllib3\util\connection.py", line 74, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did 

http://www.spindices.com/documents/indexnews/announcements/20150121-134545/134545_endocovidienbrew546.pdf


In [55]:
columns = ['type', 'name', 'ticker', 'announcement_date', 'implementation_date', 'effective_date']
df = pd.DataFrame(columns=columns)

In [32]:
def find_datetime_from_text(string):
    search_results = re.search('[A-z]+(\.?) \d\d?(\W+)\d{4}', string)
    if search_results is None:
        print('No date found in ' + string)
        return None
    matched_string = search_results.group(0)
    splitted_date_string = re.split("\W+", matched_string)
    date_string = ' '.join(splitted_date_string)
    try:
        date = datetime.strptime(date_string, '%B %d %Y')
    except:
        traceback.print_exc(file=sys.stdout)
        try:
            date = datetime.strptime(date_string, '%b %d %Y')
        except:
            traceback.print_exc(file=sys.stdout)
            date = None
    return date

In [54]:
NUMBER_OF_CHARACTERS_UNTIL_BRACKET = 20
MAX_TICKER_LENGTH = 6 # A ticker cannot have more than 5 characters

def get_ticker(stock_name, pages):
    stock_name = stock_name.replace('(', '') # remove brackets because brackets disturb the regex 
    stock_name = stock_name.replace(')', '')
    stock_name_words = list(filter(None, re.split("([A-Z]*[a-z]*)||([A-Z]+)",stock_name)))
    text = ''
    for page in pages:
        text += page.extract_text()
    text = text.replace('\n', '')
    text = strip_co_inc(text)
    number_of_words = len(stock_name_words)
    while (number_of_words > 0):
        stock_name_combinations = combinations(stock_name_words, number_of_words)
        for stock_name_combination in stock_name_combinations:
            word = ''.join(stock_name_combination)
            word = strip_co_inc(word)
            pattern = word + ".{0," + str(NUMBER_OF_CHARACTERS_UNTIL_BRACKET) + "}?\([A-Z]+\s*?:\s*?([A-z]+(;[A-z]+)*)\)"
            try: 
                matches = re.search(pattern, text, re.IGNORECASE)
            except:
                traceback.print_exc(file=sys.stdout)
                continue
            if matches is not None and (len(matches.group(1)) <= MAX_TICKER_LENGTH):
                print(matches)
#                 if stock_name == 'Evergy renamed from Westar Energy':
#                     breakpoint()
                return matches.group(1)
        number_of_words -= 1
    print('Error: Stock name '+ stock_name + ' not found in PDF!')
    print(text)
    return None
    

In [20]:
def strip_co_inc(name):
    name = name.replace(' ', '') # remove spaces
    name = name.replace('.', '').replace(',', '') # remove periods and commas
    name = name.replace('Company', '').replace('Co', '').replace('Inc', '').replace('&', '').strip()
    name = ' '.join(name.split())
    return name

In [21]:
def get_change_date_type(pages):
    '''
    If the document is referring to the day on which the change is effective before opening of trading, 
    return "effective_date".
    Otherwise, if the document is referring to the day on which the change is effective after close of trading,
    return "implementation_date".
    '''
    text = ''
    for page in pages:
        text += page.extract_text()
    text = text.replace('\n', '')
    if 'open of trading' in text or 'prior to the open' in text:
        return "effective_date"
    elif 'close of trading' in text:
        return "implementation_date"
    else:
        print("Error: Cannot determine if change date is implementation or effective date.")
        print(text)
        return None

In [56]:
for filename in os.listdir(dirPath):
    pdf = pdfplumber.open(dirPath + filename)
    announcement_date = find_datetime_from_text(pdf.pages[0].extract_text())
    change_date_type = get_change_date_type(pdf.pages)
    if change_date_type is None:
        continue
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            if any("S&P 500 INDEX" in str(item) for item in table[0]):
                print(table)
                change_date = find_datetime_from_text(table[0][0])
                last_change_type = ""
                for i in range(2, len(table)):
                    
                    change_type = table[i][0]
                    if change_type == "" or change_type == None:
                        change_type = last_change_type
                    else:
                        last_change_type = change_type
                    
                    name = table[i][1].replace('\n', '')
                    print(name)
                    ticker = get_ticker(name, pdf.pages)
                    if ticker is None:
                        continue
                    print(ticker)
                    row = [{'type': change_type, 'name': name, 'ticker': ticker, 
                            change_date_type: change_date, 'announcement_date': announcement_date}]
                    df = df.append(pd.DataFrame(row), ignore_index=True)


[['S&P 500 INDEX – September 26, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'NVR', 'Consumer Discretionary', 'Homebuilding'], ['DELETED', 'Jefferies \nFinancial Group', 'Financials', 'Multi-Sector Holdings']]
NVR
<re.Match object; span=(279, 292), match='NVR(NYSE:NVR)'>
NVR
Jefferies Financial Group
<re.Match object; span=(303, 336), match='JefferiesFinancialGroup(NYSE:JEF)'>
JEF
[['S&P 500 INDEX – October 3, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Las Vegas \nSands', 'Consumer Discretionary', 'Casinos & Gaming'], ['DELETED', 'Nektar \nTherapeutics', 'Health Care', 'Pharmaceuticals']]
Las Vegas Sands
<re.Match object; span=(216, 241), match='LasVegasSandsrp(NYSE:LVS)'>
LVS
Nektar Therapeutics
<re.Match object; span=(252, 281), match='NektarTherapeutics(NASD:NKTR)'>
NKTR
[['S&P 500 INDEX – November 21, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS S

[['S&P 500 INDEX – March 17, 2015', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Henry Schein', 'Health Care', 'Health Care Distributors'], ['DELETED', 'Carefusion', 'Health Care', 'Health Care Equipment']]
Henry Schein
<re.Match object; span=(1418, 1440), match='HenrySchein(NASD:HSIC)'>
HSIC
Carefusion
<re.Match object; span=(1451, 1473), match='Carefusionrp(NYSE:CFN)'>
CFN
[['S&P 500 INDEX – March 20, 2015', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Equinix', 'Information Technology', 'Internet Software & \nServices'], ['', 'S L Green Realty', 'Financials', 'Office REITs'], ['', 'Hanesbrands', 'Consumer Discretionary', 'Apparel, Accessories & \nLuxury Goods'], ['DELETED', 'Denbury \nResources', 'Energy', 'Oil & Gas Exploration & \nProduction'], ['', 'Nabors Industries', 'Energy', 'Oil & Gas Drilling'], ['', 'Avon Products', 'Consumer Staples', 'Personal Products']]
Equinix
<re.Match objec

[['S&P 500 INDEX – March 29, 2016', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Hologic', 'Health Care', 'Health Care Equipment'], ['', 'Centene', 'Health Care', 'Managed Health Care'], ['DELETED', 'Pepco Holdings', 'Utilities', 'Electric Utilities'], ['', 'Ensco', 'Energy', 'Oil & Gas Drilling']]
Hologic
<re.Match object; span=(204, 222), match='Hologic(NASD:HOLX)'>
HOLX
Centene
<re.Match object; span=(552, 571), match='Centenerp(NYSE:CNC)'>
CNC
Pepco Holdings
<re.Match object; span=(233, 256), match='PepcoHoldings(NYSE:POM)'>
POM
Ensco
<re.Match object; span=(582, 600), match='Enscoplc(NYSE:ESV)'>
ESV
[['S&P 500 INDEX –  April 1, 2016', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Foot Locker', 'Consumer Discretionary', 'Apparel Retail'], ['DELETED', 'Cameron \nInternational', 'Energy', 'Oil & Gas Equipment & \nServices']]
Foot Locker
<re.Match object; span=(106, 125), match='FootLocker(NYS

Traceback (most recent call last):
  File "<ipython-input-32-eefabec40daa>", line 10, in find_datetime_from_text
    date = datetime.strptime(date_string, '%B %d %Y')
  File "C:\Users\Yingchen\Anaconda3\envs\finance\lib\_strptime.py", line 577, in _strptime_datetime
    tt, fraction, gmtoff_fraction = _strptime(data_string, format)
  File "C:\Users\Yingchen\Anaconda3\envs\finance\lib\_strptime.py", line 359, in _strptime
    (data_string, format))
ValueError: time data 'FEB 23 2017' does not match format '%B %d %Y'
[['S&P 500 INDEX – FEBRUARY 28, 2017', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Incyte', 'Health Care', 'Biotechnology'], ['DELETED', 'Spectra Energy', 'Energy', 'Oil & Gas Storage & \nTransportation']]
Incyte
<re.Match object; span=(166, 182), match='yterp(NASD:INCY)'>
INCY
Spectra Energy
<re.Match object; span=(193, 217), match='SpectraEnergyrp(NYSE:SE)'>
SE
[['S&P 500 INDEX – MARCH 1, 2017', None, None, None], ['', 'COMPA

[['S&P 500 INDEX – September 18, 2017', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Cadence Design \nSystems', 'Information Technology', 'Application Software'], ['DELETED', 'Staples', 'Consumer Discretionary', 'Specialty Stores']]
Cadence Design Systems
<re.Match object; span=(217, 248), match='CadenceDesignSystems(NASD:CDNS)'>
CDNS
Staples
<re.Match object; span=(259, 277), match='Staples(NASD:SPLS)'>
SPLS
[['S&P 500 INDEX – OCTOBER 13, 2017', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Norwegian \nCruise Line', 'Consumer Discretionary', 'Hotels, Resorts & Cruise \nLines'], ['DELETED', 'Level 3 \nCommunications', 'Telecommunication Services', 'Alternative Carriers']]
Norwegian Cruise Line
<re.Match object; span=(185, 226), match='NorwegianCruiseLineHoldingsLtd(NASD:NCLH)'>
NCLH
Level 3 Communications
<re.Match object; span=(237, 266), match='Level3mmunications(NYSE:LVLT)'>
LVLT
[['S&P 500 

[['S&P 500 INDEX – MAY 31, 2018', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'ABIOMED', 'Health Care', 'Health Care Equipment'], ['DELETED', 'Wyndham \nWorldwide', 'Consumer Discretionary', 'Hotels, Resorts & Cruise \nLines']]
ABIOMED
<re.Match object; span=(195, 213), match='ABIOMED(NASD:ABMD)'>
ABMD
Wyndham Worldwide
<re.Match object; span=(224, 252), match='WyndhamWorldwiderp(NYSE:WYN)'>
WYN
[['S&P 500 INDEX – JUNE 5, 2018', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Evergy (renamed \nfrom Westar \nEnergy)', 'Utilities', 'Electric Utilities'], ['DELETED', 'Navient', 'Financials', 'Consumer Finance']]
Evergy (renamed from Westar Energy)
<re.Match object; span=(165, 186), match='WestarEnergy(NYSE:WR)'>
> <ipython-input-54-6b2e8d1d8271>(29)get_ticker()
-> return matches.group(1)
(Pdb) matches.group(1)
'WR'
(Pdb) c
WR
Navient
<re.Match object; span=(214, 234), match='Navientrp(NASD:NAVI)'>
N

[['S&P 500 INDEX – APRIL 2 , 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Dow', 'Materials', 'Commodity Chemical*']]
Dow
<re.Match object; span=(250, 263), match='Dow(NYSE:DOW)'>
DOW
[['S&P 500 INDEX – APRIL 3 , 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['DELETED', 'Brighthouse \nFinancial', 'Financials', 'Life & Health Insurance']]
Brighthouse Financial
<re.Match object; span=(336, 366), match='BrighthouseFinancial(NASD:BHF)'>
BHF
[['S&P 500 INDEX – June 3, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Corteva', 'Materials', 'Fertilizers & Agricultural \nChemicals*']]
Corteva
<re.Match object; span=(501, 517), match='rteva(NYSE:CTVA)'>
CTVA
[['S&P 500 INDEX – June 4, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['DELETED', 'Fluor', 'Industrials', 'Construction & \nEngineering']]
Fluor
<re.Mat

In [61]:
df

,type,name,ticker,announcement_date,implementation_date,effective_date
0,ADDED,NVR,NVR,2019-09-20,NaN,2019-09-26
1,DELETED,Jefferies Financial Group,JEF,2019-09-20,NaN,2019-09-26
2,ADDED,Las Vegas Sands,LVS,2019-09-26,NaN,2019-10-03
3,DELETED,Nektar Therapeutics,NKTR,2019-09-26,NaN,2019-10-03
4,ADDED,ServiceNow,NOW,2019-11-18,NaN,2019-11-21
...,...,...,...,...,...,...
228,ADDED,Ametek,AME,2013-09-11,2013-09-20 00:00:00,NaT
229,DELETED,Advanced Micro Devices,AMD,2013-09-11,2013-09-20 00:00:00,NaT
230,DELETED,SAIC,SAI,2013-09-11,2013-09-20 00:00:00,NaT
231,ADDED,Arista Networks,ANET,2018-08-23,NaN,2018-08-28


In [59]:
df.to_excel('sp500_rebalance_announcements.xlsx')